In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
base_image_path=keras.utils.get_file("sf.jpg",origin="https://img-datasets.s3.amazonaws.com/sf.jpg")
style_reference_image_path=keras.utils.get_file("starry_night.jpg",origin="https://img-datasets.s3.amazonaws.com/starry_night.jpg")

original_width,original_height=keras.utils.load_img(base_image_path).size
img_height=400
img_width=round(original_width * img_height / original_height)

In [3]:
def preprocess_image(image_path):
    img=keras.utils.load_img(
        image_path,target_size=(img_height,img_width))
    img=keras.utils.img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=keras.applications.vgg19.preprocess_input(img)
    return img

def deprocess_image(img):
    img=img.reshape((img_height,img_width,3))
    img[:,:,0] +103.939
    img[:,:,1] +116.779
    img[:,:,2] +123.68
    img=np.clip(img,0,255).astype("uint8")
    return img

In [4]:
model=keras.applications.vgg19.VGG19(weights="imagenet",include_top=False)

outputs_dict=dict([(layer.name,layer.output) for layer in model.layers])
feature_extactor=keras.Model(inputs=model.input,outputs=outputs_dict)

In [5]:
def content_loss(base_img,combination_img):
    return tf.reduce_sum(tf.square(combination_img - base_img))

In [6]:
def gram_matrix(x):
    x=tf.transpose(x,(2,0,1))
    features=tf.reshape(x,(tf.shape(x)[0],-1))
    gram=tf.matmul(features,tf.transpose(features))
    return gram

def style_loss(style_img,combination_img):
    S=gram_matrix(style_img)
    C=gram_matrix(combination_img)
    channels=3
    size=img_height * img_width
    return tf.reduce_sum(tf.square(S-C)) / (4.0 * (channels ** 2) * (size ** 2))

In [7]:
def total_variation_loss(x):
    a=tf.square(
        x[:,:img_height -1, : img_width -1, :] - x[:, 1:, : img_width - 1, :]
    )
    b=tf.square(
        x[:,:img_height -1, : img_width -1, :] - x[:, : img_height -1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))

In [8]:
style_layer_names=[
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
content_layer_name="block5_conv2"
total_variation_weight=1e-6
style_weight=1e-6
content_weight=2.5e-8

def compute_loss(combination_image,base_image,style_reference_image):
    input_tensor=tf.concat(
        [base_image,style_reference_image,combination_image],axis=0
    )
    features=feature_extactor(input_tensor)
    loss=tf.zeros(shape=())
    layer_features=features[content_layer_name]
    base_image_features=layer_features[0, :, :, :]
    combination_features=layer_features[2, :, :, :]
    loss=loss+content_weight * content_loss(
        base_image_features,combination_features
    )
    for layer_name in style_layer_names:
        layer_features=features[layer_name]
        style_reference_features=layer_features[1, :, :, :]
        combinaion_features=layer_features[2, :, :, :]
        style_loss_value=style_loss(style_reference_features,combinaion_features)
        loss += (style_weight / len(style_layer_names)) * style_loss_value

    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

In [9]:
@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))

iterations = 4000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
        print(f"Iteration {i}: loss={loss:.2f}")
        img = deprocess_image(combination_image.numpy())
        fname = f"combination_image_at_iteration_{i}.png"
        keras.utils.save_img(fname, img)

Iteration 100: loss=8136.64
Iteration 200: loss=6652.29
Iteration 300: loss=6062.85
Iteration 400: loss=5742.34
Iteration 500: loss=5536.00
Iteration 600: loss=5390.36
Iteration 700: loss=5281.30
Iteration 800: loss=5195.85
Iteration 900: loss=5126.86
Iteration 1000: loss=5070.08
Iteration 1100: loss=5022.24
Iteration 1200: loss=4981.45
Iteration 1300: loss=4946.01
Iteration 1400: loss=4915.01
Iteration 1500: loss=4887.72
Iteration 1600: loss=4863.56
Iteration 1700: loss=4842.00
Iteration 1800: loss=4822.64
Iteration 1900: loss=4805.08
Iteration 2000: loss=4789.17
Iteration 2100: loss=4774.64
Iteration 2200: loss=4761.38
Iteration 2300: loss=4749.24
Iteration 2400: loss=4738.09
Iteration 2500: loss=4727.80
Iteration 2600: loss=4718.29
Iteration 2700: loss=4709.49
Iteration 2800: loss=4701.32
Iteration 2900: loss=4693.74
Iteration 3000: loss=4686.67
Iteration 3100: loss=4680.07
Iteration 3200: loss=4673.90
Iteration 3300: loss=4668.11
Iteration 3400: loss=4662.68
Iteration 3500: loss=46